In [16]:
df = pd.read_csv("../data/processed/all_shots-v4.csv", index_col=0)
df.columns

Index(['Shot Distance', 'X Location', 'Y Location', 'Away Team', 'Season Type',
       'Year', 'Shot Zone Basic_Above the Break 3',
       'Shot Zone Basic_Backcourt', 'Shot Zone Basic_In The Paint (Non-RA)',
       'Shot Zone Basic_Left Corner 3', 'Shot Zone Basic_Mid-Range',
       'Shot Zone Basic_Restricted Area', 'Shot Zone Basic_Right Corner 3',
       'Shot Zone Area_Back Court(BC)', 'Shot Zone Area_Center(C)',
       'Shot Zone Area_Left Side Center(LC)', 'Shot Zone Area_Left Side(L)',
       'Shot Zone Area_Right Side Center(RC)', 'Shot Zone Area_Right Side(R)',
       'Shot Zone Range_16-24 ft.', 'Shot Zone Range_24+ ft.',
       'Shot Zone Range_8-16 ft.', 'Shot Zone Range_Back Court Shot',
       'Shot Zone Range_Less Than 8 ft.', 'GAME_ID', 'EVENTNUM', 'target',
       'PERIOD', 'PLAYER1_NAME', 'PLAYER1_TEAM_ABBREVIATION', 'OPPONENT_TEAM',
       'at_home', 'PREVIOUS_OFF_REBOUND', 'PREVIOUS_DEF_REBOUND',
       'PREVIOUS_OFF_TURNOVER', 'PREVIOUS_OFF_MISSED', 'PREVIOUS_EVEN

In [65]:
# Import libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from imblearn.over_sampling import RandomOverSampler

pd.set_option('display.max_columns', 500)
df = pd.read_csv("../data/processed/all_shots-v4.csv", index_col=0)
df = df[['DETAILLED_SHOT_TYPE', 'PLAYER1_NAME', 'TS%', '3PT', 'target', 'PREVIOUS_OFF_REBOUND', 'PREVIOUS_DEF_REBOUND',
       'PREVIOUS_OFF_TURNOVER', 'E_DEF_RATING', 'E_OFF_RATING', 'Shot Zone Area_Center(C)',
       'Shot Zone Area_Left Side Center(LC)', 'Shot Zone Area_Left Side(L)', 'Shot Zone Area_Right Side Center(RC)', 'Shot Zone Area_Right Side(R)',
       'Shot Zone Range_16-24 ft.', 'Shot Zone Range_24+ ft.', 'Shot Zone Range_8-16 ft.', 'Shot Zone Range_Less Than 8 ft.']]

def update_shot_type(data):    
    data.loc[(data.DETAILLED_SHOT_TYPE != 'JUMP SHOT') & (data.DETAILLED_SHOT_TYPE != "FREE THROW"), 'DETAILLED_SHOT_TYPE'] = 'OTHER'
    return data

df = update_shot_type(df)

In [66]:
X_train = ""
y_train = ""
X_test = ""
y_test = ""
y_pred = ""

def for_player(player):
    global X_test
    global X_train
    global y_test
    global y_train

    df_iverson = df[df.PLAYER1_NAME==player]
    X = df_iverson.drop(['target', 'PLAYER1_NAME'], axis=1)
    X = pd.get_dummies(X)
    y = df_iverson.target

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle=True, stratify=y)
    
    # OverSampling
    rOs = RandomOverSampler()
    X_train, y_train = rOs.fit_resample(X_train, y_train)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=50)

    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
    
    # matrice de confusion
    display(pd.crosstab(y_test, y_pred, rownames=["Classe réelle"], colnames=["Classe prédite"]))

def objective(trial):
    global X_test
    global X_train
    global y_test
    global y_train
    global y_pred
    
    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        # use exact for small dataset.
        "tree_method": "exact",
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
    }

    if param["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
        # minimum child weight, larger the term more conservative the tree.
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        # defines how selective algorithm is.
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    train = xgb.DMatrix(data=X_train, label=y_train)
    test = xgb.DMatrix(data=X_test, label=y_test)
    
    bst = xgb.train(param, train)
    preds = bst.predict(test)
    y_pred = np.rint(preds)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy


# Allen Iverson

In [67]:
for_player("Allen Iverson")

[I 2024-08-14 10:16:42,333] A new study created in memory with name: no-name-70933309-026b-4e05-ad8b-dd07ad57a77d
[I 2024-08-14 10:16:42,354] Trial 0 finished with value: 0.4705742762221167 and parameters: {'booster': 'gblinear', 'lambda': 1.0134534451492592e-05, 'alpha': 0.9500811988933371, 'subsample': 0.5864573979142533, 'colsample_bytree': 0.8581825657631539}. Best is trial 0 with value: 0.4705742762221167.
[I 2024-08-14 10:16:42,376] Trial 1 finished with value: 0.7021831988609397 and parameters: {'booster': 'gblinear', 'lambda': 0.0027869927659127133, 'alpha': 3.097114999792213e-06, 'subsample': 0.6101846462231673, 'colsample_bytree': 0.3134298634366672}. Best is trial 1 with value: 0.7021831988609397.
[I 2024-08-14 10:16:42,401] Trial 2 finished with value: 0.7021831988609397 and parameters: {'booster': 'gblinear', 'lambda': 4.595092122817064e-08, 'alpha': 9.89439697189504e-08, 'subsample': 0.6955463031783992, 'colsample_bytree': 0.8242459189618514}. Best is trial 1 with value: 

Number of finished trials:  50
Best trial:
  Value: 0.7045562411010916
  Params: 
    booster: dart
    lambda: 1.0868551776192758e-05
    alpha: 3.91312838862173e-08
    subsample: 0.2422516095153323
    colsample_bytree: 0.9761004664057835
    max_depth: 5
    min_child_weight: 3
    eta: 5.478225179105559e-06
    gamma: 0.0001381477894270714
    grow_policy: lossguide
    sample_type: uniform
    normalize_type: forest
    rate_drop: 8.029461632292187e-06
    skip_drop: 3.5165299259072084e-06


Classe prédite,0.0,1.0
Classe réelle,,
0,1335,648
1,618,1613


# Stephen Curry

In [68]:
for_player("Stephen Curry")

[I 2024-08-14 10:16:48,358] A new study created in memory with name: no-name-484523df-70de-4a33-bf2d-b0a0de5a6650
[I 2024-08-14 10:16:48,433] Trial 0 finished with value: 0.6772100567721006 and parameters: {'booster': 'dart', 'lambda': 0.0180163330330982, 'alpha': 0.17283844600457973, 'subsample': 0.4308245865696405, 'colsample_bytree': 0.5745732350563497, 'max_depth': 9, 'min_child_weight': 7, 'eta': 4.310349151087109e-05, 'gamma': 7.896138775513855e-05, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 3.3143621255051286e-05, 'skip_drop': 5.330304405872379e-08}. Best is trial 0 with value: 0.6772100567721006.
[I 2024-08-14 10:16:48,455] Trial 1 finished with value: 0.6909975669099757 and parameters: {'booster': 'gblinear', 'lambda': 0.028682760389765766, 'alpha': 3.595730233722702e-08, 'subsample': 0.5899737052629206, 'colsample_bytree': 0.465044306510488}. Best is trial 1 with value: 0.6909975669099757.
[I 2024-08-14 10:16:48,518] Trial 2 

Number of finished trials:  50
Best trial:
  Value: 0.6920789402541228
  Params: 
    booster: gblinear
    lambda: 2.738223155181145e-06
    alpha: 1.207744325061958e-07
    subsample: 0.6335947210156785
    colsample_bytree: 0.3167896324973301


Classe prédite,0.0,1.0
Classe réelle,,
0,1090,549
1,687,1373


# LeBron James

In [69]:
for_player("LeBron James")

[I 2024-08-14 10:16:52,403] A new study created in memory with name: no-name-debaf53a-827e-431f-ba0b-16b5b446e624
[I 2024-08-14 10:16:52,447] Trial 0 finished with value: 0.6859070464767616 and parameters: {'booster': 'gblinear', 'lambda': 2.7907480314344366e-08, 'alpha': 1.1257558628879145e-05, 'subsample': 0.47674061518373273, 'colsample_bytree': 0.4143194334668356}. Best is trial 0 with value: 0.6859070464767616.
[I 2024-08-14 10:16:52,554] Trial 1 finished with value: 0.6861569215392304 and parameters: {'booster': 'dart', 'lambda': 0.0003437819785142613, 'alpha': 1.4703301647606897e-06, 'subsample': 0.2685655229004656, 'colsample_bytree': 0.5440885741668604, 'max_depth': 5, 'min_child_weight': 7, 'eta': 1.8963634877639824e-07, 'gamma': 1.3842348034561496e-06, 'grow_policy': 'lossguide', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 2.9752964750027237e-05, 'skip_drop': 5.333975523567324e-06}. Best is trial 1 with value: 0.6861569215392304.
[I 2024-08-14 10:16:52,6

Number of finished trials:  50
Best trial:
  Value: 0.68928035982009
  Params: 
    booster: dart
    lambda: 0.1482883569708423
    alpha: 0.0023112367310208983
    subsample: 0.9584242255000498
    colsample_bytree: 0.6680911860930804
    max_depth: 9
    min_child_weight: 6
    eta: 0.0030256144169056214
    gamma: 0.0005016291089895771
    grow_policy: depthwise
    sample_type: uniform
    normalize_type: tree
    rate_drop: 0.00015196710542421718
    skip_drop: 5.167741537168002e-05


Classe prédite,0.0,1.0
Classe réelle,,
0,2130,1309
1,1187,3378


# Kobe Bryant

In [70]:
for_player("Kobe Bryant")

[I 2024-08-14 10:17:01,658] A new study created in memory with name: no-name-ff45c3ab-7ce8-4ac4-854d-2296b829a8f4
[I 2024-08-14 10:17:01,703] Trial 0 finished with value: 0.7042412286881781 and parameters: {'booster': 'gblinear', 'lambda': 2.1145092967933105e-07, 'alpha': 8.597797365808222e-06, 'subsample': 0.8954351097007247, 'colsample_bytree': 0.20641528490661232}. Best is trial 0 with value: 0.7042412286881781.
[I 2024-08-14 10:17:01,823] Trial 1 finished with value: 0.7018458503593068 and parameters: {'booster': 'dart', 'lambda': 0.15719303862482445, 'alpha': 0.018213306392810515, 'subsample': 0.26445790296782595, 'colsample_bytree': 0.3139162604044837, 'max_depth': 9, 'min_child_weight': 5, 'eta': 0.0009030456406355114, 'gamma': 3.649480983944453e-05, 'grow_policy': 'lossguide', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 1.7714586603080304e-06, 'skip_drop': 0.023142442513325097}. Best is trial 0 with value: 0.7042412286881781.
[I 2024-08-14 10:17:01,907] T

Number of finished trials:  50
Best trial:
  Value: 0.7048048471185008
  Params: 
    booster: gblinear
    lambda: 1.2915341036428298e-08
    alpha: 7.206660762863064e-05
    subsample: 0.9935812506468631
    colsample_bytree: 0.5094815746903443


Classe prédite,0.0,1.0
Classe réelle,,
0,2208,960
1,1190,2739


# Dirk Nowitzki

In [71]:
for_player("Dirk Nowitzki")

[I 2024-08-14 10:17:07,309] A new study created in memory with name: no-name-c0ddc5c8-4731-45f4-be05-fc5365b7ff94
[I 2024-08-14 10:17:07,382] Trial 0 finished with value: 0.6532208588957055 and parameters: {'booster': 'dart', 'lambda': 0.43502485228437576, 'alpha': 0.00041282323966781943, 'subsample': 0.25392131950005353, 'colsample_bytree': 0.3515600728770905, 'max_depth': 3, 'min_child_weight': 7, 'eta': 0.07280829097109241, 'gamma': 5.2185159156076484e-05, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 0.004851900366517042, 'skip_drop': 0.002048638113355668}. Best is trial 0 with value: 0.6532208588957055.
[I 2024-08-14 10:17:07,423] Trial 1 finished with value: 0.4237730061349693 and parameters: {'booster': 'gblinear', 'lambda': 7.619843683325605e-08, 'alpha': 0.4541804089934232, 'subsample': 0.9870016953135763, 'colsample_bytree': 0.7298221780828011}. Best is trial 0 with value: 0.6532208588957055.
[I 2024-08-14 10:17:07,508] Trial 2

Number of finished trials:  50
Best trial:
  Value: 0.6562883435582823
  Params: 
    booster: dart
    lambda: 2.8529289401209378e-05
    alpha: 8.784649295887119e-05
    subsample: 0.8783149029656637
    colsample_bytree: 0.32090613349969316
    max_depth: 3
    min_child_weight: 6
    eta: 0.07149480713205097
    gamma: 1.516371845468953e-05
    grow_policy: depthwise
    sample_type: uniform
    normalize_type: forest
    rate_drop: 0.9900821423091148
    skip_drop: 0.08911532351835606


Classe prédite,0.0,1.0
Classe réelle,,
0,2196,567
1,1703,2054


# Dwyane Wade 

In [72]:
for_player("Dwyane Wade")

[I 2024-08-14 10:17:25,616] A new study created in memory with name: no-name-80613d3b-6257-4a0d-be1f-4bf6f5c25a2a
[I 2024-08-14 10:17:25,646] Trial 0 finished with value: 0.6685827195976289 and parameters: {'booster': 'gblinear', 'lambda': 0.012357977946566892, 'alpha': 0.01050302457598988, 'subsample': 0.8973798781463489, 'colsample_bytree': 0.5064857954301459}. Best is trial 0 with value: 0.6685827195976289.
[I 2024-08-14 10:17:25,716] Trial 1 finished with value: 0.6693012394467397 and parameters: {'booster': 'dart', 'lambda': 0.002733486841724965, 'alpha': 3.809249858603545e-07, 'subsample': 0.9902102795638019, 'colsample_bytree': 0.24982060029436265, 'max_depth': 5, 'min_child_weight': 8, 'eta': 8.099349277741993e-07, 'gamma': 7.780943628103804e-08, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 9.859911645077958e-05, 'skip_drop': 0.0013859867787690502}. Best is trial 1 with value: 0.6693012394467397.
[I 2024-08-14 10:17:25,787] Trial

Number of finished trials:  50
Best trial:
  Value: 0.6730734686545716
  Params: 
    booster: dart
    lambda: 3.9867514217454393e-05
    alpha: 9.837059112769506e-08
    subsample: 0.4868558177228015
    colsample_bytree: 0.5058937020929823
    max_depth: 5
    min_child_weight: 4
    eta: 0.2527222864061563
    gamma: 2.112980165806881e-08
    grow_policy: depthwise
    sample_type: weighted
    normalize_type: forest
    rate_drop: 0.003987975840841047
    skip_drop: 2.4942762980191427e-08


Classe prédite,0.0,1.0
Classe réelle,,
0,1478,960
1,891,2238


# Paul Pierce

In [73]:
for_player("Paul Pierce")

[I 2024-08-14 10:17:33,198] A new study created in memory with name: no-name-89fad0e0-5858-439c-8d41-1d5168240f8c
[I 2024-08-14 10:17:33,259] Trial 0 finished with value: 0.6809402332361516 and parameters: {'booster': 'gbtree', 'lambda': 0.0015062357133781056, 'alpha': 3.010876830104278e-05, 'subsample': 0.9879988907748902, 'colsample_bytree': 0.20509737518903137, 'max_depth': 7, 'min_child_weight': 10, 'eta': 0.00011686612024460278, 'gamma': 0.06432841957930999, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 0.6809402332361516.
[I 2024-08-14 10:17:33,325] Trial 1 finished with value: 0.6909620991253644 and parameters: {'booster': 'gbtree', 'lambda': 3.6870937571390296e-07, 'alpha': 5.771326035807176e-06, 'subsample': 0.863354265242088, 'colsample_bytree': 0.5916610437105414, 'max_depth': 5, 'min_child_weight': 3, 'eta': 0.00035577882595941085, 'gamma': 0.5541178425100881, 'grow_policy': 'depthwise'}. Best is trial 1 with value: 0.6909620991253644.
[I 2024-08-14 10:17:33,412]

Number of finished trials:  50
Best trial:
  Value: 0.6924198250728864
  Params: 
    booster: gblinear
    lambda: 2.119819931620608e-06
    alpha: 0.000296981984166381
    subsample: 0.6388902669354078
    colsample_bytree: 0.5230999459522406


Classe prédite,0.0,1.0
Classe réelle,,
0,1591,843
1,850,2204


# Tim Duncan

In [74]:
for_player("Tim Duncan")

[I 2024-08-14 10:17:37,813] A new study created in memory with name: no-name-bfa6ca7e-b98d-471b-9570-cafbce770fce
[I 2024-08-14 10:17:37,902] Trial 0 finished with value: 0.6655013799448022 and parameters: {'booster': 'dart', 'lambda': 1.220350675453576e-07, 'alpha': 4.9754787609151494e-05, 'subsample': 0.43996764103217156, 'colsample_bytree': 0.8097393506379016, 'max_depth': 5, 'min_child_weight': 3, 'eta': 0.00022747596305949924, 'gamma': 7.07652027457247e-08, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 0.006256553007727671, 'skip_drop': 0.3571624930765526}. Best is trial 0 with value: 0.6655013799448022.
[I 2024-08-14 10:17:37,942] Trial 1 finished with value: 0.6638454461821527 and parameters: {'booster': 'gblinear', 'lambda': 0.009839479412592337, 'alpha': 0.0007423397479554105, 'subsample': 0.6937749651403586, 'colsample_bytree': 0.6273201939512765}. Best is trial 0 with value: 0.6655013799448022.
[I 2024-08-14 10:17:38,033] Tria

Number of finished trials:  50
Best trial:
  Value: 0.6673413063477461
  Params: 
    booster: dart
    lambda: 0.26174352836259157
    alpha: 5.194416522481589e-08
    subsample: 0.7333992882044708
    colsample_bytree: 0.9970460884981013
    max_depth: 3
    min_child_weight: 6
    eta: 0.002487557602306387
    gamma: 0.0004369611821928495
    grow_policy: lossguide
    sample_type: uniform
    normalize_type: forest
    rate_drop: 0.019325303051792257
    skip_drop: 0.03784114947726144


Classe prédite,0.0,1.0
Classe réelle,,
0,1264,1134
1,676,2361


# Russell Westbrook

In [75]:
for_player("Russell Westbrook")

[I 2024-08-14 10:17:47,348] A new study created in memory with name: no-name-91546a2b-b3b8-415b-88c3-bcdab211cfae
[I 2024-08-14 10:17:47,378] Trial 0 finished with value: 0.6530651711467801 and parameters: {'booster': 'gblinear', 'lambda': 3.9672708334252926e-07, 'alpha': 5.5630283510267997e-08, 'subsample': 0.22710303426056067, 'colsample_bytree': 0.4801215358596486}. Best is trial 0 with value: 0.6530651711467801.
[I 2024-08-14 10:17:47,484] Trial 1 finished with value: 0.6604138464513634 and parameters: {'booster': 'gbtree', 'lambda': 0.005030202528261699, 'alpha': 0.5712602577129596, 'subsample': 0.8574659834121652, 'colsample_bytree': 0.8680582509514567, 'max_depth': 7, 'min_child_weight': 7, 'eta': 0.0001462421476001719, 'gamma': 2.429820343047201e-06, 'grow_policy': 'depthwise'}. Best is trial 1 with value: 0.6604138464513634.
[I 2024-08-14 10:17:47,589] Trial 2 finished with value: 0.6567395087990717 and parameters: {'booster': 'gbtree', 'lambda': 1.3434154547058829e-08, 'alpha

Number of finished trials:  50
Best trial:
  Value: 0.664088184103655
  Params: 
    booster: gbtree
    lambda: 0.1553067226609043
    alpha: 9.40374424621823e-06
    subsample: 0.42296504304333504
    colsample_bytree: 0.9196525912745434
    max_depth: 7
    min_child_weight: 8
    eta: 0.0002509194582208215
    gamma: 0.00011316904210459366
    grow_policy: lossguide


Classe prédite,0.0,1.0
Classe réelle,,
0,1548,868
1,871,1884


# James Harden

In [76]:
for_player("James Harden")

[I 2024-08-14 10:17:57,151] A new study created in memory with name: no-name-065220bf-adc8-48bc-a01b-e52c36a443f9
[I 2024-08-14 10:17:57,244] Trial 0 finished with value: 0.6903124385930438 and parameters: {'booster': 'gbtree', 'lambda': 0.0006011248661312587, 'alpha': 0.0024255164135970893, 'subsample': 0.5880291880417152, 'colsample_bytree': 0.8528576002845532, 'max_depth': 5, 'min_child_weight': 4, 'eta': 0.0008053346379071104, 'gamma': 5.2594229382369815e-08, 'grow_policy': 'lossguide'}. Best is trial 0 with value: 0.6903124385930438.
[I 2024-08-14 10:17:57,283] Trial 1 finished with value: 0.6677146787188053 and parameters: {'booster': 'gblinear', 'lambda': 4.9546083793820295e-08, 'alpha': 4.0527377390279514e-07, 'subsample': 0.5647170597133129, 'colsample_bytree': 0.6721186271014662}. Best is trial 0 with value: 0.6903124385930438.
[I 2024-08-14 10:17:57,369] Trial 2 finished with value: 0.6659461583808214 and parameters: {'booster': 'gbtree', 'lambda': 0.45583386549064653, 'alph

Number of finished trials:  50
Best trial:
  Value: 0.6952249950874435
  Params: 
    booster: gbtree
    lambda: 1.0852501924593756e-06
    alpha: 1.7338599320685001e-07
    subsample: 0.5345273745486468
    colsample_bytree: 0.9204779094744939
    max_depth: 9
    min_child_weight: 8
    eta: 5.18562267859712e-06
    gamma: 0.023924013061419093
    grow_policy: lossguide


Classe prédite,0.0,1.0
Classe réelle,,
0,1747,377
1,1252,1713


# Kevin Durant  

In [77]:
for_player("Kevin Durant")

[I 2024-08-14 10:18:04,123] A new study created in memory with name: no-name-99968152-63aa-4d37-b7f1-1a9a9e51753a
[I 2024-08-14 10:18:04,164] Trial 0 finished with value: 0.6976006345429308 and parameters: {'booster': 'gblinear', 'lambda': 2.3315396289261827e-07, 'alpha': 0.004068696957297376, 'subsample': 0.45697637988288076, 'colsample_bytree': 0.22190899379445173}. Best is trial 0 with value: 0.6976006345429308.
[I 2024-08-14 10:18:04,207] Trial 1 finished with value: 0.701764822526274 and parameters: {'booster': 'gblinear', 'lambda': 0.006829057343696372, 'alpha': 0.09166689393654334, 'subsample': 0.8939893185908987, 'colsample_bytree': 0.20509997536030983}. Best is trial 1 with value: 0.701764822526274.
[I 2024-08-14 10:18:04,331] Trial 2 finished with value: 0.6930398572278406 and parameters: {'booster': 'dart', 'lambda': 0.0004497226272072256, 'alpha': 1.540547741846339e-05, 'subsample': 0.6822583835154129, 'colsample_bytree': 0.4706504431093128, 'max_depth': 7, 'min_child_weigh

Number of finished trials:  50
Best trial:
  Value: 0.7043426531826293
  Params: 
    booster: gblinear
    lambda: 0.00012691551501936953
    alpha: 0.023537359194423693
    subsample: 0.8415875720430429
    colsample_bytree: 0.7289899256415275


Classe prédite,0.0,1.0
Classe réelle,,
0,1350,627
1,916,2150


# Kevin Garnett

In [78]:
for_player("Kevin Garnett")

[I 2024-08-14 10:18:08,649] A new study created in memory with name: no-name-905e6851-5227-4ba3-8b2a-a7e52b80ec61
[I 2024-08-14 10:18:08,737] Trial 0 finished with value: 0.6674915635545556 and parameters: {'booster': 'dart', 'lambda': 6.105309445341015e-06, 'alpha': 0.0004301230411549669, 'subsample': 0.35797732805638866, 'colsample_bytree': 0.4632494019648209, 'max_depth': 7, 'min_child_weight': 4, 'eta': 0.028658964490485675, 'gamma': 0.007957691151214061, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'forest', 'rate_drop': 2.0241499161795348e-05, 'skip_drop': 4.972497558215535e-07}. Best is trial 0 with value: 0.6674915635545556.
[I 2024-08-14 10:18:08,773] Trial 1 finished with value: 0.6697412823397075 and parameters: {'booster': 'gblinear', 'lambda': 0.0001170326120559564, 'alpha': 1.6936051960734365e-06, 'subsample': 0.483818819924558, 'colsample_bytree': 0.9127627776538418}. Best is trial 1 with value: 0.6697412823397075.
[I 2024-08-14 10:18:08,887] 

Number of finished trials:  50
Best trial:
  Value: 0.6724409448818898
  Params: 
    booster: gbtree
    lambda: 0.001958252383351005
    alpha: 0.2103421129482442
    subsample: 0.5315227503157514
    colsample_bytree: 0.9909716546191799
    max_depth: 5
    min_child_weight: 7
    eta: 0.00012068731560844416
    gamma: 0.14506346675087786
    grow_policy: depthwise


Classe prédite,0.0,1.0
Classe réelle,,
0,1234,685
1,779,1747


# Dwight Howard 

In [79]:
for_player("Dwight Howard")

[I 2024-08-14 10:18:15,116] A new study created in memory with name: no-name-dba00ec8-de82-44fe-8bf3-66c0eeca0fb1
[I 2024-08-14 10:18:15,152] Trial 0 finished with value: 0.5693034238488784 and parameters: {'booster': 'gblinear', 'lambda': 0.04644907658956135, 'alpha': 2.780949222323749e-05, 'subsample': 0.5759535363057307, 'colsample_bytree': 0.7626551377169422}. Best is trial 0 with value: 0.5693034238488784.
[I 2024-08-14 10:18:15,233] Trial 1 finished with value: 0.5669421487603306 and parameters: {'booster': 'dart', 'lambda': 6.540181879006207e-05, 'alpha': 0.0014880374757934235, 'subsample': 0.32970348662044446, 'colsample_bytree': 0.4803440865511996, 'max_depth': 3, 'min_child_weight': 10, 'eta': 7.157578572995324e-07, 'gamma': 0.0004764431166667478, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 0.026677818155271104, 'skip_drop': 0.25119834204067}. Best is trial 0 with value: 0.5693034238488784.
[I 2024-08-14 10:18:15,324] Trial 2 

Number of finished trials:  50
Best trial:
  Value: 0.6014167650531287
  Params: 
    booster: dart
    lambda: 0.017422756190815427
    alpha: 2.149029356583651e-06
    subsample: 0.5175079622662463
    colsample_bytree: 0.8919728653118968
    max_depth: 7
    min_child_weight: 9
    eta: 1.3481267108687433e-05
    gamma: 5.530571523480241e-06
    grow_policy: depthwise
    sample_type: uniform
    normalize_type: tree
    rate_drop: 9.65092659722923e-05
    skip_drop: 0.07684148779228762


Classe prédite,0.0,1.0
Classe réelle,,
0,932,868
1,854,1581


# Shaquille O'Neal

In [80]:
for_player("Shaquille O'Neal")

[I 2024-08-14 10:18:21,978] A new study created in memory with name: no-name-a8e2b956-8a0a-4516-83af-1c01eb94f1a0
[I 2024-08-14 10:18:22,029] Trial 0 finished with value: 0.6067024128686327 and parameters: {'booster': 'gbtree', 'lambda': 5.974027850777452e-07, 'alpha': 2.2038691531949532e-08, 'subsample': 0.22580887389445678, 'colsample_bytree': 0.5602688074815625, 'max_depth': 3, 'min_child_weight': 4, 'eta': 5.958180535489395e-07, 'gamma': 0.08018633864349203, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 0.6067024128686327.
[I 2024-08-14 10:18:22,059] Trial 1 finished with value: 0.4428954423592493 and parameters: {'booster': 'gblinear', 'lambda': 0.006650362333671747, 'alpha': 0.24979791453622158, 'subsample': 0.39561564476343947, 'colsample_bytree': 0.3607296253019797}. Best is trial 0 with value: 0.6067024128686327.
[I 2024-08-14 10:18:22,132] Trial 2 finished with value: 0.6080428954423592 and parameters: {'booster': 'dart', 'lambda': 5.750218960745721e-06, 'alpha': 0

Number of finished trials:  50
Best trial:
  Value: 0.6372654155495978
  Params: 
    booster: dart
    lambda: 1.2667857800352736e-05
    alpha: 3.7930332345492414e-07
    subsample: 0.8081973530637964
    colsample_bytree: 0.698458759219081
    max_depth: 7
    min_child_weight: 9
    eta: 0.018732261319120837
    gamma: 0.0004613559363222417
    grow_policy: depthwise
    sample_type: uniform
    normalize_type: forest
    rate_drop: 0.0005633415888625842
    skip_drop: 0.5627417229185924


Classe prédite,0.0,1.0
Classe réelle,,
0,968,684
1,683,1395


# Steve Nash   

In [81]:
for_player("Steve Nash")

[I 2024-08-14 10:18:29,022] A new study created in memory with name: no-name-1e2f7a5f-d586-4258-8a67-d34ce7c0a5ac
[I 2024-08-14 10:18:29,089] Trial 0 finished with value: 0.6039443905593275 and parameters: {'booster': 'dart', 'lambda': 0.06753078471938383, 'alpha': 2.0614228250003845e-07, 'subsample': 0.2319824168073013, 'colsample_bytree': 0.22602655149550124, 'max_depth': 9, 'min_child_weight': 8, 'eta': 0.011394118888984447, 'gamma': 0.6214081129726973, 'grow_policy': 'lossguide', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 3.2829816809078835e-08, 'skip_drop': 1.6368451399982242e-06}. Best is trial 0 with value: 0.6039443905593275.
[I 2024-08-14 10:18:29,156] Trial 1 finished with value: 0.6450048496605237 and parameters: {'booster': 'dart', 'lambda': 0.0010250383339236052, 'alpha': 0.004681005379920543, 'subsample': 0.6425406117270078, 'colsample_bytree': 0.748362345128061, 'max_depth': 3, 'min_child_weight': 6, 'eta': 0.6697259940422303, 'gamma': 3.386192292

Number of finished trials:  50
Best trial:
  Value: 0.6611703847397349
  Params: 
    booster: gblinear
    lambda: 0.001155756804714411
    alpha: 0.006868067525477859
    subsample: 0.3790348182801346
    colsample_bytree: 0.3292450206446355


Classe prédite,0.0,1.0
Classe réelle,,
0,938,354
1,702,1099


# Giannis Antetokounmpo

In [82]:
for_player("Giannis Antetokounmpo")

[I 2024-08-14 10:18:32,612] A new study created in memory with name: no-name-5a00afa6-f9cb-490b-9e94-5ec3b8969ef4
[I 2024-08-14 10:18:32,693] Trial 0 finished with value: 0.671280276816609 and parameters: {'booster': 'dart', 'lambda': 0.0009109638408293157, 'alpha': 4.5130204907013594e-05, 'subsample': 0.6236430252947345, 'colsample_bytree': 0.9117117590259944, 'max_depth': 7, 'min_child_weight': 5, 'eta': 0.18066087417590054, 'gamma': 1.3570800535369218e-06, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 5.024565360111002e-06, 'skip_drop': 3.138498173980975e-08}. Best is trial 0 with value: 0.671280276816609.
[I 2024-08-14 10:18:32,718] Trial 1 finished with value: 0.6789696270665129 and parameters: {'booster': 'gblinear', 'lambda': 0.0017917192204221838, 'alpha': 0.00023933779364313158, 'subsample': 0.42846360071802975, 'colsample_bytree': 0.6235651091213148}. Best is trial 1 with value: 0.6789696270665129.
[I 2024-08-14 10:18:32,771] Tri

Number of finished trials:  50
Best trial:
  Value: 0.6797385620915033
  Params: 
    booster: gblinear
    lambda: 1.2455053792919338e-05
    alpha: 2.26977011038227e-06
    subsample: 0.45775141074826636
    colsample_bytree: 0.4374050303884513


Classe prédite,0.0,1.0
Classe réelle,,
0,533,533
1,365,1170


# Kawhi Leonard

In [83]:
for_player("Kawhi Leonard")

[I 2024-08-14 10:18:36,691] A new study created in memory with name: no-name-870fc79c-b25e-434a-9d9e-4efaff6d2120
[I 2024-08-14 10:18:36,747] Trial 0 finished with value: 0.6773063448812767 and parameters: {'booster': 'gbtree', 'lambda': 0.0036560067342076914, 'alpha': 1.623930020392045e-07, 'subsample': 0.466206495392786, 'colsample_bytree': 0.6638264055739493, 'max_depth': 5, 'min_child_weight': 9, 'eta': 0.005941534307259634, 'gamma': 0.00024717549352633214, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 0.6773063448812767.
[I 2024-08-14 10:18:36,860] Trial 1 finished with value: 0.6356558972362787 and parameters: {'booster': 'dart', 'lambda': 0.34283949310916934, 'alpha': 1.2595498200469191e-05, 'subsample': 0.8089921129206905, 'colsample_bytree': 0.9786220047060621, 'max_depth': 9, 'min_child_weight': 5, 'eta': 2.8082724151869092e-08, 'gamma': 4.6452884586506234e-06, 'grow_policy': 'lossguide', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 0.0001573360

Number of finished trials:  50
Best trial:
  Value: 0.6792526274815103
  Params: 
    booster: gbtree
    lambda: 1.2057128545955205e-08
    alpha: 1.8308911209094896e-07
    subsample: 0.38645572144101137
    colsample_bytree: 0.70951984695604
    max_depth: 7
    min_child_weight: 10
    eta: 0.003421173984336536
    gamma: 0.016869276677322805
    grow_policy: lossguide


Classe prédite,0.0,1.0
Classe réelle,,
0,721,344
1,498,1006


# Jimmy Butler

In [84]:

for_player("Jimmy Butler")

[I 2024-08-14 10:18:41,533] A new study created in memory with name: no-name-9e37efa8-9069-4223-8388-d6a7e57e0007
[I 2024-08-14 10:18:41,557] Trial 0 finished with value: 0.6800636689216076 and parameters: {'booster': 'gblinear', 'lambda': 3.1750405107704017e-06, 'alpha': 0.001482450266134295, 'subsample': 0.6304817127045046, 'colsample_bytree': 0.24856205023123457}. Best is trial 0 with value: 0.6800636689216076.
[I 2024-08-14 10:18:41,583] Trial 1 finished with value: 0.6732988460007958 and parameters: {'booster': 'gblinear', 'lambda': 0.00038441896664299875, 'alpha': 6.975340327216927e-06, 'subsample': 0.7782266339487565, 'colsample_bytree': 0.46761724884624123}. Best is trial 0 with value: 0.6800636689216076.
[I 2024-08-14 10:18:41,640] Trial 2 finished with value: 0.693991245523279 and parameters: {'booster': 'dart', 'lambda': 0.022070506970893466, 'alpha': 4.181500416219316e-07, 'subsample': 0.9084085211024906, 'colsample_bytree': 0.3456031779193839, 'max_depth': 5, 'min_child_we

Number of finished trials:  50
Best trial:
  Value: 0.6999602069239952
  Params: 
    booster: gbtree
    lambda: 0.00016063474714449034
    alpha: 0.005552207631419762
    subsample: 0.7682728638423334
    colsample_bytree: 0.6002783200602098
    max_depth: 7
    min_child_weight: 5
    eta: 2.3245918092617503e-05
    gamma: 1.2733881737415545e-08
    grow_policy: depthwise


Classe prédite,0.0,1.0
Classe réelle,,
0,677,381
1,385,1070


# Chris Webber

In [85]:
for_player("Chris Webber")

[I 2024-08-14 10:18:46,728] A new study created in memory with name: no-name-7e203f81-3b5d-4380-893e-7366dc31a506
[I 2024-08-14 10:18:46,791] Trial 0 finished with value: 0.6381888082016233 and parameters: {'booster': 'dart', 'lambda': 0.04163014396081464, 'alpha': 3.172442151066232e-07, 'subsample': 0.5790763120659208, 'colsample_bytree': 0.8250387513881818, 'max_depth': 5, 'min_child_weight': 6, 'eta': 8.557316599143069e-05, 'gamma': 1.904881867885025e-08, 'grow_policy': 'lossguide', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 1.5910750527847411e-06, 'skip_drop': 0.00045392316723206404}. Best is trial 0 with value: 0.6381888082016233.
[I 2024-08-14 10:18:46,820] Trial 1 finished with value: 0.6416061512174285 and parameters: {'booster': 'gblinear', 'lambda': 0.0005048644067019971, 'alpha': 4.787076784153585e-07, 'subsample': 0.7501039699323555, 'colsample_bytree': 0.8526161640506651}. Best is trial 1 with value: 0.6416061512174285.
[I 2024-08-14 10:18:46,886] Tri

Number of finished trials:  50
Best trial:
  Value: 0.6416061512174285
  Params: 
    booster: gblinear
    lambda: 0.0005048644067019971
    alpha: 4.787076784153585e-07
    subsample: 0.7501039699323555
    colsample_bytree: 0.8526161640506651


Classe prédite,0.0,1.0
Classe réelle,,
0,704,448
1,391,798


# Jason Kidd 

In [86]:
for_player("Jason Kidd")

[I 2024-08-14 10:18:49,926] A new study created in memory with name: no-name-95c299e0-df53-458d-94c6-d3cba62fe83c
[I 2024-08-14 10:18:49,941] Trial 0 finished with value: 0.6513409961685823 and parameters: {'booster': 'gblinear', 'lambda': 0.6215737708350084, 'alpha': 0.0003918131751756557, 'subsample': 0.574362556160853, 'colsample_bytree': 0.44286014684388997}. Best is trial 0 with value: 0.6513409961685823.
[I 2024-08-14 10:18:49,968] Trial 1 finished with value: 0.6561302681992337 and parameters: {'booster': 'gbtree', 'lambda': 3.8241560936083033e-07, 'alpha': 0.0022954925510834958, 'subsample': 0.3015065175057454, 'colsample_bytree': 0.4621811488434667, 'max_depth': 3, 'min_child_weight': 6, 'eta': 0.04032399144466733, 'gamma': 0.4413587239899151, 'grow_policy': 'depthwise'}. Best is trial 1 with value: 0.6561302681992337.
[I 2024-08-14 10:18:50,016] Trial 2 finished with value: 0.6417624521072797 and parameters: {'booster': 'dart', 'lambda': 1.0052375061964384e-05, 'alpha': 0.083

Number of finished trials:  50
Best trial:
  Value: 0.6590038314176245
  Params: 
    booster: gbtree
    lambda: 1.1714248453540787e-08
    alpha: 0.08775863285638597
    subsample: 0.8223356577685508
    colsample_bytree: 0.6264056795071957
    max_depth: 3
    min_child_weight: 2
    eta: 5.461377520476724e-06
    gamma: 3.968924296591277e-07
    grow_policy: depthwise


Classe prédite,0.0,1.0
Classe réelle,,
0,432,124
1,236,252
